In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
67,application_1598227897403_0031,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
def aml_classifier(n_layers,n_neurons,l_rate):
    
    import os
    import sys
    import uuid
    import random
    
    import numpy as np
    
    from tensorflow import keras
    import tensorflow as tf
    from tensorflow.io import FixedLenFeature
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.callbacks import TensorBoard
    from tensorflow.keras import backend as K

    import math
    from hops import tensorboard

    from hops import model as hops_model
    from hops import hdfs, featurestore

    import pydoop.hdfs as pydoop
    

    batch_size=32
    num_epochs=20
    num_classes = 1
    
    # Define input function
    def create_tf_dataset(n_epochs, b_size):
        dataset_dir = featurestore.get_training_dataset_path("embeddings_training_dataset",
                                                                featurestore=featurestore.project_featurestore(),
                                                                training_dataset_version=featurestore.get_latest_training_dataset_version("embeddings_training_dataset")
                                                            )
    
        input_files = tf.io.gfile.glob(dataset_dir.replace("hopsfs","hdfs") + "/part-r-*")
        dataset = tf.data.TFRecordDataset(input_files)
        tf_record_schema = featurestore.get_training_dataset_tf_record_schema("embeddings_training_dataset")
        feature_names = feature_names = [feat.name for feat in tf_record_schema]
        feature_names = [feat for feat in tf_record_schema]
        label_name = "label"
        feature_names.remove(label_name)

        def decode(example_proto):
            example = tf.io.parse_single_example(example_proto, tf_record_schema)
            x = []
            for feature_name in feature_names:
                if feature_name != "label" and feature_name != "id":
                    x.append(example[feature_name])                
            y = [tf.cast(example[label_name], tf.float32)]
            if len(x) == 1:
                x = x[0]
            else:
                x = tf.stack(x)            
            return x,y

        dataset = dataset.map(decode).shuffle(1000).batch(b_size).repeat(n_epochs)
        return dataset

    # Define a Keras Model.
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(n_neurons, activation='relu', input_shape=(128,)))
    for _ in n_layers:
        model.add(tf.keras.layers.Dense(n_neurons, activation='relu'))

    # Compile the model.
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer= tf.keras.optimizers.Adam(l_rate),
                  metrics=['accuracy']
                 )
      
    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=tensorboard.logdir()),
        tf.keras.callbacks.ModelCheckpoint(filepath=tensorboard.logdir()),
    ]
    model.fit(create_tf_dataset(num_epochs, batch_size), 
#        verbose=0,
        epochs=num_epochs, 
        steps_per_epoch=10,
        validation_data=create_tf_dataset(num_epochs, batch_size),
        validation_steps=1,                    
        callbacks=callbacks
    )
    
    score = model.evaluate(create_tf_dataset(num_epochs, batch_size), steps=1)


    export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
    print('Exporting trained model to: {}'.format(export_path))
    
    tf.saved_model.save(model, export_path)

    print('Done exporting!')
    
    metrics = metrics={'accuracy': score[1]}
    
    hops_model.export(export_path, "aml", metrics=metrics)    
    
    return metrics

In [3]:
from hops import hdfs
import json
best_hyperparams_path = "Resources/aml_ff_best_hp.json
best_hyperparams = json.loads(hdfs.load(best_hyperparams_path))
args_dict = {}
for key in best_hyperparams.keys():
    args_dict[key] = [best_hyperparams[key]]
    
from hops import experiment
from hops import hdfs

experiment.launch(aml_classifier, args_dict, name='aml classifier', local_logdir=True, metric_key='accuracy')

Finished Experiment 

('hdfs://rpc.namenode.service.consul:8020/Projects/paysim/Experiments/application_1598227897403_0031_1', {'accuracy': 0.975, 'log': 'Experiments/application_1598227897403_0031_1/output.log'})